In [17]:
import os
import time

import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split

from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS
from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
    confusion_matrix
)
from classification.utils.utils import accuracy

import tensorflow as tf
from keras import layers, models
import pandas as pd
import seaborn as sns

dataset = Dataset()

fm_dir = "data/feature_matrices/"
model_dir = "data/models/"

nclass = dataset.nclass
naudio = dataset.naudio
classnames = dataset.list_classes()
num_classes = len(classnames)

In [ ]:
for i in range(10):

    print("Iteration: ", i)

    np.random.seed(int(time.time()))

    print("seed = ", int(time.time()))
    
    # 'echo', 'time_shift', 'scaling', 'pitch_shift', 

    myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=1.5, data_aug=['time_shift'])

    feature_length = len(myds[classnames[0], 0])
    data_aug_factor = myds.data_aug_factor


    X_aug = np.zeros((data_aug_factor * nclass * naudio, 20, 20, 1))
    y_aug = np.zeros(data_aug_factor * nclass * naudio)

    for s in range(data_aug_factor):
        for idx in range(dataset.naudio):
            for class_idx, classname in enumerate(classnames):
                y_aug[s* nclass * naudio + class_idx * naudio + idx] = class_idx
                featvec = myds[classname, idx]
                X_aug[ s* nclass * naudio + class_idx * naudio + idx, :, :] = featvec.reshape(20, 20, 1)

    np.save(fm_dir + "feature_matrix_2D_aug.npy", X_aug)
    np.save(fm_dir + "labels_aug.npy", y_aug)

    X_aug = np.load(fm_dir + "feature_matrix_2D_aug.npy")
    y_aug = np.load(fm_dir + "labels_aug.npy")

    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=0.9, stratify=y_aug
    )
    
    """
    model = models.Sequential([
        layers.Input(shape=(20, 20, 1)),
        layers.Normalization(),
        layers.Conv2D(128, kernel_size=(4, 4), activation='leaky_relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Conv2D(256, kernel_size=(4, 4), activation='leaky_relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(256, activation='leaky_relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    model.summary()
    """
    model = models.load_model(model_dir + "two.keras")
    
    ### change learining rate
    learning_rate = 0.1 * 10**(-np.log(i+1))
    
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), shuffle=True)

    # store the model

    model.save(model_dir + "two.keras", zipped=True)

    # evaluate the model

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test loss: {test_loss}')

    # plot the confusion matrix
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    plt.figure(figsize=(3, 3))
    confmat = confusion_matrix(y_test, y_pred_classes)
    sns.heatmap(confmat.T, square=True, annot=True, fmt="d", cbar=False, xticklabels=classnames, yticklabels=classnames)
    plt.xlabel("True label", fontsize=8)
    plt.ylabel("Predicted label", fontsize=8)
    plt.tight_layout()
    plt.savefig(f"plots/confusion_matrix.pdf")
    plt.show()

    
    print(f'Test accuracy: {test_acc}')

     #calculate the f1 score
    from sklearn.metrics import f1_score
    f1 = f1_score(y_test, y_pred_classes, average='weighted')

    # calculate the percentage for each class
    class_percentages = np.mean(y_pred, axis=0) * 100
    for classname, percentage in zip(classnames, class_percentages):
        print(f'{classname}: {percentage:.2f}%')

In [ ]:
### play all the sounds
audio_util = AudioUtil()

for classname in classnames:
    for idx in range(40):
        path = dataset.__getitem__((classname, idx))
        audio = audio_util.open(path)
        audio_util.play(audio)
        time.sleep(5)
        print(f'Playing {classname} {idx}')

In [ ]:
# open the prediction file

data = pd.read_csv('C:\LELEC210X\LELEC210X\predictions.csv')

# plot the confusion matrix

y_pred = data['prediction']
for i in range(len(y_pred)):
    y_pred[i] = [float(x) for x in y_pred[i].replace('[', '').replace(']', '').split()]
    y_pred[i] = classnames[np.argmax(y_pred[i])]
y_test = np.repeat(classnames, 134)
y_pred_mean = data['mean_prediction']
for i in range(len(y_pred_mean)):
    y_pred_mean[i] = [float(x) for x in y_pred_mean[i].replace('[', '').replace(']', '').split()]
    y_pred_mean[i] = classnames[np.argmax(y_pred_mean[i])]
y_pred_weighted = data['weighted_prediction']
for i in range(len(y_pred_weighted)):
    y_pred_weighted[i] = [float(x) for x in y_pred_weighted[i].replace('[', '').replace(']', '').split()]
    y_pred_weighted[i] = classnames[np.argmax(y_pred_weighted[i])]
    
sns.set(font_scale=0.8)

plt.figure(figsize=(3, 3))
confmat = confusion_matrix(y_test, y_pred[:-1])
sns.heatmap(confmat.T, square=True, annot=True, fmt="d", cbar=False, xticklabels=classnames, yticklabels=classnames)
plt.xlabel("True label", fontsize=8)
plt.ylabel("Predicted label", fontsize=8)
plt.tight_layout()
plt.savefig(f"plots/confusion_matrix_direct_predictions.pdf")
plt.show()

plt.figure(figsize=(3, 3))
confmat = confusion_matrix(y_test, y_pred_mean[:-1])
sns.heatmap(confmat.T, square=True, annot=True, fmt="d", cbar=False, xticklabels=classnames, yticklabels=classnames)
plt.xlabel("True label", fontsize=8)
plt.ylabel("Predicted label", fontsize=8)
plt.tight_layout()
plt.savefig(f"plots/confusion_matrix_mean_predictions.pdf")
plt.show()

plt.figure(figsize=(3, 3))
confmat = confusion_matrix(y_test, y_pred_weighted[:-1])
sns.heatmap(confmat.T, square=True, annot=True, fmt="d", cbar=False, xticklabels=classnames, yticklabels=classnames)
plt.xlabel("True label", fontsize=8)
plt.ylabel("Predicted label", fontsize=8)
plt.tight_layout()
plt.savefig(f"plots/confusion_matrix_weighted_predictions.pdf")
plt.show()

# print the accuracy

print(f'Accuracy for the direct predictions: {accuracy(y_pred[:-1], y_test)}')
print(f'Accuracy for the mean predictions: {accuracy(y_pred_mean[:-1], y_test)}')
print(f'Accuracy for the weighted predictions: {accuracy(y_pred_weighted[:-1], y_test)}')

In [ ]:
# open the prediction file

data_new = pd.read_csv('C:\LELEC210X\LELEC210X\predictions_three_keras.csv')

# plot the confusion matrix

y_pred = data_new['prediction']
for i in range(len(y_pred)):
    y_pred[i] = [float(x) for x in y_pred[i].replace('[', '').replace(']', '').split()]
    y_pred[i] = classnames[np.argmax(y_pred[i])]
y_test = np.repeat(classnames, 658/5)
y_pred_mean = data_new['mean_prediction']
for i in range(len(y_pred_mean)):
    y_pred_mean[i] = [float(x) for x in y_pred_mean[i].replace('[', '').replace(']', '').split()]
    y_pred_mean[i] = classnames[np.argmax(y_pred_mean[i])]
y_pred_weighted = data_new['weighted_prediction']
for i in range(len(y_pred_weighted)):
    y_pred_weighted[i] = [float(x) for x in y_pred_weighted[i].replace('[', '').replace(']', '').split()]
    y_pred_weighted[i] = classnames[np.argmax(y_pred_weighted[i])]


plt.figure(figsize=(3, 3))
confmat = confusion_matrix(y_test, y_pred[:-4])
sns.heatmap(confmat.T, square=True, annot=True, fmt="d", cbar=False, xticklabels=classnames, yticklabels=classnames)
plt.xlabel("True label", fontsize=8)
plt.ylabel("Predicted label", fontsize=8)
plt.tight_layout()
plt.savefig(f"plots/confusion_matrix_prev_predictions.pdf")
plt.show()

# print the accuracy

print(f'Accuracy for the predictions: {accuracy(y_pred[:-4], y_test)}')
print(f'Accuracy for the mean predictions: {accuracy(y_pred_mean[:-4], y_test)}')
print(f'Accuracy for the weighted predictions: {accuracy(y_pred_weighted[:-4], y_test)}')